In [16]:
from PTO import random, solve

### Solving TSP with GRASP emulated in PTO

We'll use the same problem generator and fitness as when we solved TSP directly in PTO.

In [54]:
n = 10
def randprob(n): 
    c = [[0 for x in range(n)] for x in range(n)] # initialise cost matrix
    for i in range(n):
        for j in range(n):
            if i == j:
                c[i][j] = 0 # zero diagonal
            elif i > j:
                c[i][j] = c[j][i] # symmetric matrix
            else:
                c[i][j] = random.expovariate(1) # more interesting than uniform weights
    return c
instance = randprob(n)

In [52]:
def _fitness(perm, inst):
    # note negative indexing trick to include final step (return to origin)
    return -sum([inst[perm[i-1]][perm[i]] for i in range(0,len(perm))])
fitness = lambda perm: _fitness(perm, instance)

Next, we use the same generic GRASP generator as when we solved the ORDERING problem with GRASP emulated in PTO.

In [55]:
def randsol():
  solution = empty_solution()
  while(not complete(solution)):
    #print(solution)
    features = allowed_features(solution)
    costs = {feat:cost_feature(solution, feat) for feat in features}
    min_cost, max_cost = min(costs.values()), max(costs.values())
    RCL = [feat for feat in features if costs[feat] <= min_cost + alpha * (max_cost - min_cost)]
    #print(RCL)
    selected_feature = random.choice(RCL) # only source of randomness
    solution = add_feature(solution, selected_feature)
  return solution 

### TSP-specific functions for GRASP in PTO

Finally, we fill in the functions used by `randsol` in a way appropriate to TSP. In fact, all of them are the same as in the ORDERING problem, except for the `cost_feature` functions and a small bookkeeping change in `allowed_features` (we will now count from 0 to n-1 instead of 1 to n).

In [51]:
n=len(instance)

def empty_solution():
  return []

def complete(solution):
  return len(solution)==n

def allowed_features(solution):
  all_items = range(n) # count from 0 to n-1
  remaining_items = [item for item in all_items if item not in solution]
  return remaining_items

def cost_feature(solution, feat):
  if len(solution) == 0:
    # all cities cost nothing as start city. 
    # NB this will give a uniform random choice of start city,
    # which is better than hardcoding it!
    return 0 
  last_item = solution[-1]
  dist = instance[last_item][feat]
  return dist

def add_feature(solution, feat):
  sol = solution[:] + [feat]
  return sol

Now we can quickly test our approach without metaheuristic search. Again, we observe that the fully greedy approach is the best in this scenario.

In [53]:
alpha = 0.0 # completely greedy

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))
    
print("===")    
    
alpha = 0.5 # half way

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))
    
print("===")    
    
alpha = 1.0 # completely random

for i in range(5):
    x = randsol()
    print("Random solution: fitness %.3f; %s" % (fitness(x), str(x)))

Random solution: fitness -3.774; [1, 8, 7, 5, 6, 4, 3, 0, 9, 2]
Random solution: fitness -4.027; [7, 8, 1, 2, 6, 5, 3, 4, 9, 0]
Random solution: fitness -4.027; [7, 8, 1, 2, 6, 5, 3, 4, 9, 0]
Random solution: fitness -4.485; [2, 6, 5, 3, 4, 8, 7, 0, 9, 1]
Random solution: fitness -4.027; [0, 7, 8, 1, 2, 6, 5, 3, 4, 9]
===
Random solution: fitness -7.167; [9, 8, 6, 2, 1, 4, 5, 3, 0, 7]
Random solution: fitness -7.566; [5, 0, 9, 8, 4, 1, 2, 6, 7, 3]
Random solution: fitness -6.255; [5, 9, 2, 6, 7, 1, 8, 4, 3, 0]
Random solution: fitness -4.847; [9, 7, 6, 5, 0, 1, 8, 4, 3, 2]
Random solution: fitness -7.659; [5, 6, 2, 4, 3, 1, 9, 7, 8, 0]
===
Random solution: fitness -16.852; [1, 3, 9, 2, 5, 7, 4, 0, 8, 6]
Random solution: fitness -11.255; [9, 8, 5, 2, 7, 3, 1, 6, 4, 0]
Random solution: fitness -10.988; [1, 4, 2, 6, 0, 8, 7, 9, 3, 5]
Random solution: fitness -10.109; [8, 4, 3, 7, 2, 1, 5, 6, 9, 0]
Random solution: fitness -13.887; [1, 9, 3, 6, 8, 7, 0, 2, 4, 5]


Now we can test with metaheuristic optimisation. This time, we observe that a less greedy approach in the generator ($\alpha=0.5$) typically improves performance, at least with the HC solver.

In [67]:
print("alpha solver -fitness")
for alpha in [0, 0.5, 1.0]:
    for solver in ["RS", "HC", "EA"]:
        ind, fit = solve(randsol, fitness, solver=solver, budget=150)
        print(alpha, solver, fit)


alpha solver -fitness
0 RS -3.004062971092426
0 HC -3.004062971092426
0 EA -3.004062971092426
0.5 RS -2.818138494676305
0.5 HC -2.8460228197790927
0.5 EA -3.7527285496458207
1.0 RS -3.8050944504310973
1.0 HC -4.438373785243424
1.0 EA -3.8785901485701535
